<a href="https://colab.research.google.com/github/FabioBoccia/Progetto_ESM/blob/generation-hyperparameters-tuning/generate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taming Transformers

This notebook is a minimal working example to generate faces images as in [Taming Transformers for High-Resolution Image Synthesis](https://github.com/CompVis/taming-transformers).

## Setup
The setup code in this section was written to be [run in a Colab environment](https://colab.research.google.com/github/CompVis/taming-transformers/blob/master/scripts/taming-transformers.ipynb).
It should be run only once per runtime.
We first clone the repository and upload the model checkpoint and config from Google Drive

In [1]:
%cd /content
%rm -r sample_data
!git clone https://github.com/CompVis/taming-transformers
%cd taming-transformers
!mkdir -p logs

from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/2021-04-23T18-19-01_ffhq_transformer logs/

/content
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1335, done.
remote: Total 1335 (delta 0), reused 0 (delta 0), pack-reused 1335
Receiving objects: 100% (1335/1335), 409.77 MiB | 16.42 MiB/s, done.
Resolving deltas: 100% (277/277), done.
/content/taming-transformers
Mounted at /content/drive


Next, we install required dependencies.

In [2]:
!pip install --upgrade omegaconf einops transformers pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 4.2 MB 64.6 MB/s 
     |████████████████████████████████| 584 kB 61.1 MB/s 
     |████████████████████████████████| 596 kB 61.1 MB/s 
     |████████████████████████████████| 117 kB 72.9 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 74.4 MB/s 
     |████████████████████████████████| 409 kB 51.5 MB/s 
     |████████████████████████████████| 140 kB 75.8 MB/s 
     |████████████████████████████████| 1.1 MB 46.4 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 73.2 MB/s 
     |████████████████████████████████| 144 kB 77.5 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=496257dd9018e3348827fae412ad5e3c78d

## Loading the model

We load and print the config.

In [3]:
import sys
%cd /content/taming-transformers
sys.path.append(".")
from omegaconf import OmegaConf
config_path = "/content/taming-transformers/logs/2021-04-23T18-19-01_ffhq_transformer/configs/2021-04-23T18-19-01-project.yaml"
config = OmegaConf.load(config_path)
import yaml
print(yaml.dump(OmegaConf.to_container(config)))

/content/taming-transformers
data:
  params:
    batch_size: 24
    num_workers: 24
    train:
      params:
        size: 256
      target: taming.data.faceshq.FFHQTrain
    validation:
      params:
        size: 256
      target: taming.data.faceshq.FFHQValidation
  target: cutlit.DataModuleFromConfig
model:
  base_learning_rate: 0.0625
  params:
    cond_stage_config: __is_unconditional__
    first_stage_config:
      params:
        ddconfig:
          attn_resolutions:
          - 16
          ch: 128
          ch_mult:
          - 1
          - 1
          - 2
          - 2
          - 4
          double_z: false
          dropout: 0.0
          in_channels: 3
          num_res_blocks: 2
          out_ch: 3
          resolution: 256
          z_channels: 256
        embed_dim: 256
        lossconfig:
          target: taming.modules.losses.vqperceptual.DummyLoss
        n_embed: 1024
      target: taming.models.vqgan.VQModel
    first_stage_key: image
    transformer_config:
   

Instantiate the model and load the checkpoint.

In [4]:
from taming.models.cond_transformer import Net2NetTransformer
import torch
model = Net2NetTransformer(**config.model.params)
ckpt_path = "/content/taming-transformers/logs/2021-04-23T18-19-01_ffhq_transformer/checkpoints/last.ckpt"
sd = torch.load(ckpt_path, map_location="cuda")["state_dict"]
missing, unexpected = model.load_state_dict(sd, strict=False)

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
Using no cond stage. Assuming the training is intended to be unconditional. Prepending 0 as a sos token.


In [5]:
model.cuda().eval()
torch.set_grad_enabled(False)

## Tuning Hyperparameters


In [6]:
%cd /content
!wget --no-check-certificate --user=corso --password=p2021corso http://www.grip.unina.it/download/corso/ffhq_real.zip
!unzip -q ffhq_real.zip
#!rm -d 0_real
#!rm ffhq_real.zip
%mkdir -p /content/ffhq/real
%mkdir -p /content/ffhq/synthesized

/content
--2022-05-26 15:46:10--  http://www.grip.unina.it/download/corso/ffhq_real.zip
Resolving www.grip.unina.it (www.grip.unina.it)... 143.225.28.237
Connecting to www.grip.unina.it (www.grip.unina.it)|143.225.28.237|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.grip.unina.it/download/corso/ffhq_real.zip [following]
--2022-05-26 15:46:11--  https://www.grip.unina.it/download/corso/ffhq_real.zip
Connecting to www.grip.unina.it (www.grip.unina.it)|143.225.28.237|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="corso"
Reusing existing connection to www.grip.unina.it:443.
HTTP request sent, awaiting response... 200 OK
Length: 4072216854 (3.8G) [application/zip]
Saving to: ‘ffhq_real.zip’

ffhq_real.zip       100%[===================>]   3.79G  14.8MB/s    in 4m 28s  

2022-05-26 15:50:41 (14.5 MB/s) - ‘ffhq_real.zip’

In [16]:
%cd /content/taming-transformers
from scripts.sample_fast import run
from tqdm import tqdm
from PIL import Image
import numpy as np
import os 
import tensorflow as tf
%cd /content

/content/taming-transformers
/content


In [86]:
## 
batch_size = 3

trainmodel = tf.keras.models.load_model('/content/drive/MyDrive/my_keras_model.h5')

logdir="/content/ffhq/synthesized/"

def loss(k, p, temp):
    %rm /content/ffhq/synthesized/*
    %cd /content/ffhq/real/
    %mv * /content/0_real/
    %cd /content/0_real/
    %mv $(ls | shuf -n $num_samples) ../ffhq/real/
    run(logdir, model, batch_size, temp, k, unconditional=model.be_unconditional, num_samples=num_samples, top_p=p)
    dataset = tf.keras.utils.image_dataset_from_directory(
        '/content/ffhq/',
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=5,
        image_size=(256, 256),
        shuffle=True,
        validation_split=0,
        interpolation='bilinear',
        crop_to_aspect_ratio=True,
    )
    loss, AUC, accuracy = trainmodel.evaluate(dataset)
    print('||tested with top_k:', k, '- top_p:',p,'- temperature: ',t)
    return AUC

In [105]:
# minimize AUC (loss function)
import csv

num_samples = 1
ks = np.arange(220, 421, 20)
ps = np.arange(0.8, 1.0, 0.05)
ts = np.arange(0.85, 1.15, 0.05)


%cd /content/drive/MyDrive
for n in range(500):
    k = int(np.random.choice(ks))
    p = np.random.choice(ps)
    t = np.random.choice(ts)
    l = loss(k, p, t)
    print("n:",n+1)
    test = (k, p, t, l)
    with open ('/content/drive/MyDrive/export.csv','a',newline = '') as csvfile:
        my_writer = csv.writer(csvfile, delimiter = ',')
        my_writer.writerow(test)
        csvfile.close()

/content/drive/MyDrive
rm: cannot remove '/content/ffhq/synthesized/*': No such file or directory
/content/ffhq/real
/content/0_real
Running in unconditional sampling mode, producing 1 samples.


Sampling: 1it [00:04,  4.50s/it]


Found 2 files belonging to 2 classes.
1/1 [==============================] - 0s 108ms/step - loss: 6.0216e-07 - auc: 1.0000 - accuracy: 1.0000
||tested with top_k: 260 - top_p: 0.8 - temperature:  1.0500000000000003
n: 1
/content/ffhq/real
/content/0_real
Running in unconditional sampling mode, producing 1 samples.


Sampling: 1it [00:04,  4.43s/it]


Found 2 files belonging to 2 classes.
1/1 [==============================] - 0s 109ms/step - loss: 4.8850e-04 - auc: 1.0000 - accuracy: 1.0000
||tested with top_k: 400 - top_p: 0.8 - temperature:  0.85
n: 2
/content/ffhq/real
/content/0_real
Running in unconditional sampling mode, producing 1 samples.


Sampling: 1it [00:04,  4.46s/it]


Found 2 files belonging to 2 classes.
1/1 [==============================] - 0s 112ms/step - loss: 2.1511e-04 - auc: 1.0000 - accuracy: 1.0000
||tested with top_k: 400 - top_p: 0.9000000000000001 - temperature:  0.9500000000000001
n: 3
/content/ffhq/real
/content/0_real
Running in unconditional sampling mode, producing 1 samples.


Sampling: 0it [00:02, ?it/s]


KeyboardInterrupt: ignored

## Generate image samples
Generated images will be saved in `logdir/samples/top_k_{top_k}_top_p_{top_p}/`
as PNGs

if you wish to display the generated images instead of saving them in logdir, 
set save = False


In [ ]:
num_samples = 1000
batch_size = 3
top_k = 300
top_p = 1.0
temperature = 1.0

save = True
logdir="/content/drive/MyDrive/"

if (not a.isEmpty):
    top_k = a[0].k
    top_p = a[0].p 
    temperature = a[0].t

In [ ]:
from scripts.sample_fast import sample_unconditional
from tqdm import tqdm
from PIL import Image
import numpy as np

def show_image(s):
  s = s.detach().cpu().numpy().transpose(0,2,3,1)[0]
  s = ((s+1.0)*127.5).clip(0,255).astype(np.uint8)
  s = Image.fromarray(s)
  display(s)

@torch.no_grad()
def run_live(model, batch_size, temperature, top_k, unconditional=True, num_samples=50000, top_p=None):
    print(f"Running in unconditional sampling mode, producing {num_samples} samples.")
    batches = [batch_size for _ in range(num_samples//batch_size)] + [num_samples % batch_size]
    for n, bs in tqdm(enumerate(batches), desc="Sampling"):
        if bs == 0: break
        logs = sample_unconditional(model, batch_size=bs, temperature=temperature, top_k=top_k, top_p=top_p)
        show_image(logs["samples"])

In [ ]:
from scripts.sample_fast import run
import os 

if save == True:
    logdir = os.path.join(logdir, "samples", f"top_k_{top_k}_top_p_{top_p}")
    print(f"Logging to {logdir}")
    os.makedirs(logdir, exist_ok=True)
    run(logdir, model, batch_size, temperature, top_k, unconditional=model.be_unconditional, num_samples=num_samples, top_p=top_p)
else:
    print(f"Generating with top_k = {top_k}, top_p = {top_p}")
    run_live(model, batch_size, temperature, top_k, unconditional=model.be_unconditional, num_samples=num_samples, top_p=top_p)

Logging to /content/drive/MyDrive/samples/top_k_300_top_p_1.0
Running in unconditional sampling mode, producing 3000 samples.


Sampling: 196it [41:39, 12.72s/it]